# Prerequisites
Before running this notebook, ensure the following:

### Python version 
Python 3.9 or later is recommended for compatibility and performance.
### System requirements
A machine equipped with a modern NVIDIA GPU is strongly recommended for efficient prediction (e.g., NVIDIA T4, V100, or A100). 

    Note: If your local machine does not have a suitable GPU, you can use free cloud services like Google Colab, which occasionally provides access to GPUs such as Tesla T4. See the optional section below for how to check and use Colab’s GPU resources.

## (Optional) Guide to Changing the Runtime to Use a T4 GPU in Google Colab
**Use the GPU for accelerate the runtime of prediction**

Google Colab provides access to different types of hardware accelerators, including GPUs. Follow this guide to ensure you're using a **T4 GPU** for your machine learning tasks.

## Step-by-Step Instructions

### 1. Open the Colab Notebook

- Start by opening your Colab notebook or create a new one by going to [Google Colab](https://colab.research.google.com/).

### 2. Access Runtime Settings

- At the top of the notebook interface, click on the **Runtime** tab.
- In the dropdown menu, select **Change runtime type**.

### 3. Select GPU as the Hardware Accelerator

- In the "Runtime type" window, look for the **Hardware accelerator** option.
- From the dropdown menu, select **GPU** (if it's not already selected).

### 4. Confirm GPU Type (T4 GPU)

Google Colab provides different types of GPUs, and T4 is one of the options. To confirm that your environment is using a **T4 GPU**, follow these steps:

1. Run the following code in a new code cell to check the current GPU type:

    ```python
    !nvidia-smi
    ```

    If a **T4** GPU is available, the output will display **Tesla T4**. If a different type of GPU is shown (e.g., P100 or K80), try resetting your runtime or wait for a new session to assign a T4.

2. You can reset the runtime by selecting **Runtime** > **Factory reset runtime** or by disconnecting and reconnecting from the **Runtime** tab.

### 5. Verify the T4 GPU Usage

Once you have confirmed you're using a T4 GPU, run this command again to ensure you're connected:

```python
!nvidia-smi


In [ ]:
# If you are not using Google Colab, you do not need to run this, please proceed directly to the next step (Cloning the repository).
!nvidia-smi
%pwd

# BerlinHUYoloStomata Repository

This repository contains the BerlinHUYoloStomata project, which involves research and development on the Yolo model for stomata detection in wheat plants.

## Cloning the Repository

To clone this repository to your local machine, follow these steps:

### Prerequisites

Ensure you have **Git** installed on your system. You can verify it by running the following command in your terminal:
``` bash
git --version
```
If Git is installed correctly, you will see an output like:
``` bash
git version 2.42.0
```
If you see an error such as command not found, you can download and install Git from the official website: https://git-scm.com/downloads

Colab has git built-in.

In [ ]:
!git clone https://github.com/rayneuro/BerlinHUYoloStomata.git

In [ ]:

%ls
%cd BerlinHUYoloStomata

### Run Prediction

To perform predictions using the trained model, update the image or folder path in the code below.

**Remember to change 'PATH_TO_YOUR_IMAGE_OR_FOLDER' to your actual path!**

Your predictions will be saved in ultralytics/runs/obb/PREDICTION_FOLDER_NAME/

In [ ]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('./ultralytics/runs/obb/BerlinHU-T22Without2_RayEn/weights/best.pt')

# Run prediction
model.predict(
    source='PATH_TO_YOUR_IMAGE_OR_FOLDER',  # Replace this with your actual image or folder path
    name='PREDICTION_FOLDER_NAME',          # Output folder name under runs/obb/PREDICTION_FOLDER_NAME
    save=True,                              # Save predicted images
    save_txt=True,                          # Save bounding box coordinates in YOLO txt format
    save_json=True,                         # Save predictions in COCO JSON format
    save_conf=True,                         # Save confidence scores
    show_conf=False                         # Do not show confidence on image
)

# Converting YOLOv8 OBB Format `.txt` Files to `.csv`

This guide explains how to convert YOLOv8 Oriented Bounding Box (OBB) format `.txt` files into a `.csv` file format for easier analysis or processing.

## YOLOv8 OBB `.txt` File Format

In the YOLOv8 OBB `.txt` format, each line in the file corresponds to a detected object in an image, with the following structure:

Where:
- `class_id`: The class label of the detected object.
- `x1, y1`, `x2, y2`, `x3, y3`, `x4, y4`: The coordinates of the four corners of the oriented bounding box.
- `confidence`: The confidence score of the detection.


Convert to
- `center x` , `center y` : The center of the rotated bounding box
- `w` , `h` : the width and height of rotated bounding box  
- `angle` : the angle of rotate



In [ ]:
%ls

import os
import pandas as pd
import math
import numpy as np

# Function to convert the coordinate to cx ,cy , h , w
def get_angle(x1,y1,x2,y2,x3,y3,x4,y4):
    if x2-x1 != 0 and x4-x3 != 0:
        theta = (math.atan((y2-y1)/(x2-x1)) + math.atan((y4-y3)/(x4-x3)))/2
    else:
        theta = np.pi/2

    if theta >= 0:
        angle = theta
    else:
        angle = theta + np.pi
    return angle,theta


def get_w_and_h(x1,y1,x2,y2,x3,y3,x4,y4):
    w = math.sqrt((x2-x1)**2+(y2-y1)**2)
    h = math.sqrt((x1-x3)**2+(y1-y3)**2)
    return w,h

def get_x_and_y(x1,y1,x2,y2,x3,y3,x4,y4,theta,w,h):
    cx = (x2+x3)/2
    cy = (y2+y3)/2
    x = cx - w/2
    y = cy - h/2
    return x,y,cx,cy


# labels = [os.path.join('./runs/obb/predict/labels', x) for x in os.listdir('./runs/obb/predict/labels') if x[-3:] == "txt"]
labels = [os.path.join('/home/oscar/Stomata/BerlinHUYoloStomata/datasets/labels/val', x) for x in os.listdir('/home/oscar/Stomata/BerlinHUYoloStomata/datasets/labels/val') if x[-3:] == "txt"]
#labelsname = [f for f in os.listdir('./runs/obb/predict/labels')]

labels.sort()
#labelsname.sort()

# Convert and save the annotations

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 ,
                           "hair":4
                           } # human hair

id_to_class_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))


df = pd.DataFrame(columns=[ 'class' ,'boundingbox_x' , 'boundingbox_y','boundingbox_width' ,'boundingbox_height' , 'angle' , 'confidence', 'File Name' ])

# Read txt label file
for name in labels:
    fr_line = open(name)
    for bbox_line in fr_line.readlines():
            info = bbox_line.strip().split(' ')
            cls= int(info[0])
            x1,y1,x2,y2,x3,y3,x4,y4 = float(info[1]) , float(info[2]) , float(info[3]) , float(info[4]) , float(info[5]) , float(info[6]) , float(info[7]) , float(info[8])
            conf = float(info[8])
            file_name = name.split('/')[-1]
            angle,theta = get_angle(x1,y1,x2,y2,x3,y3,x4,y4)
            w,h = get_w_and_h(x1,y1,x2,y2,x3,y3,x4,y4)
            x,y,cx,cy = get_x_and_y(x1,y1,x2,y2,x3,y3,x4,y4,theta,w,h)
            new_row = {'class': id_to_class_name_mapping[cls] , 'boundingbox_x' : cx, 'boundingbox_y' : cy , 'boundingbox_width' : w, 'boundingbox_height' : h, 'angle': angle, 'confidence' : conf , 'File Name' : file_name }
            df.loc[len(df)] = new_row



# Save the Dataframe to csv file
save_name = './Results.csv'
df.to_csv(save_name , encoding = 'utf-8' ,index = False)


# Converting Ground Truth files (.xml to .txt)
If your ground truth annotations are still in XML format and not yet converted to YOLO .txt, you must run this script first as YOLO models require .txt labels in the proper format for both training and evaluation.

To convert your annotated XML files into YOLO-compatible `.txt` format with rotated bounding boxes, please make sure that your files are organized as the instructions below in your working directory.

#### Prepare the Folder Structure

Make sure your directories are organized as follows:
``` bash
Xml_to_Yolotxt/
├── xml/        # Place your XML annotation files here
└── Yolotxt/    # The output .txt files will be saved here
```

In [ ]:
# Get the list of files in the directory 
fr = os.listdir('./Xml_to_Yolotxt/xml')

In [ ]:
#coding:utf-8


"""
<object>
    <type>robndbox</type>
    <name>wenben</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <robndbox>
        <cx>860.5666</cx>
        <cy>734.5734</cy>
        <w>644.8657</w>
        <h>52.3775</h>
        <angle>3.031593</angle>
    </robndbox>
</object>
cx and cy represent the coordinates of the center point of the bndbox (the upper left corner of the coordinate system is the origin, 
the right is the positive x direction, and the downward is the positive y direction); h and w are the height and width of the character block; 
angle is the rotation angle information. It should be noted here that the rotation angle value obtained by the roLabelImg annotation is as follows: 
first, draw a horizontal bndbox, angle = 0 at this time. If you rotate clockwise, the angle value obtained is a positive value of a radian unit. 
According to this idea, if you rotate counterclockwise, the angle value obtained should be a negative value of a radian unit, but this is not the case. 
For example, in the example above, the actual angle should be rotated a little counterclockwise, but the angle value obtained is 3.081593, 
which is also a positive value. Assuming that the angle is negative when rotating counterclockwise, then its angle theta should be: theta = angle-pi,
"""

import os
import numpy as np

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import sys

import math


def rotate(angle, x, y):
    """
    
    :param angle:   radian
    :param x:       x
    :param y:       y
    :return:
    """
    rotatex = math.cos(angle) * x - math.sin(angle) * y
    rotatey = math.cos(angle) * y + math.sin(angle) * x
    return rotatex, rotatey

def xy_rorate(theta, x, y, centerx, centery):
    """
    
    :param theta:
    :param x:
    :param y:
    :param centerx:
    :param centery:
    :return:
    """
    r_x, r_y = rotate(theta, x - centerx, y - centery)
    return centerx+r_x, centery+r_y

def rec_rotate(x, y, width, height, theta):
    """
    
    :param x:
    :param y:
    :param width:
    :param height:
    :param theta:
    :return:
    """
    centerx = x + width / 2
    centery = y + height / 2

    x1, y1 = xy_rorate(theta, x, y, centerx, centery)
    x2, y2 = xy_rorate(theta, x+width, y, centerx, centery)
    x3, y3 = xy_rorate(theta, x, y+height, centerx, centery)
    x4, y4 = xy_rorate(theta, x+width, y+height, centerx, centery)

    return x1, y1, x2, y2, x4, y4,x3, y3


def test(x,y,cx,cy,theta):
    x1_test = cx+(x-cx)*math.cos(theta)-(y-cy)*math.sin(theta)
    y1_test = cy+(y-cy)*math.cos(theta)+(x-cx)*math.sin(theta)
    print('x1_testx1_testx1_testx1_testx1_test',x1_test)
    print('y1_testy1_testy1_testy1_testy1_test',y1_test)


PATH = './Xml_to_Yolotxt/'



class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                "blurry.complete": 2,
                "blurry.incomplete" :3 ,
                "hair":4
                } # 


xml_path = os.path.join(PATH +'/xml/')
#print(xml_path)
txt_path = os.path.join(PATH + '/Yolotxt/')
# print(xml_path)

for line in fr:
    if line:
        tree = ET.parse(os.path.join(xml_path,line.strip()))     # open xml file
        root = tree.getroot()         # get root node
        filename = root.find('filename').text
        file_object = open(os.path.join(txt_path,filename + ".txt"), 'w') # write txt file
        # file_object_log = open(filename + ".log", 'w') #write log file
        flag = False

    for size in root.findall('size'): #
        width = int(size.find('width').text)   #
        height = int(size.find('height').text)   #

    for object in root.findall('object'): #
        name = object.find('name').text
        robndbox = object.find('robndbox')
        print(filename)
        cx = float(robndbox.find('cx').text)
        cy = float(robndbox.find('cy').text)
        w = float(robndbox.find('w').text)
        h = float(robndbox.find('h').text)
        angle = float(robndbox.find('angle').text)

        x = cx - w/2
        y = cy - h/2
        if angle<1.57:
            theta = round(angle, 6)
        else:
            theta = round(angle - np.pi, 6)
        x1, y1, x2, y2, x4, y4,x3, y3 = rec_rotate(x, y, w, h, theta)
        x1,y1,x2,y2,x4,y4,x3,y3 = int(x1) / width,int(y1) / height,int(x2) / width ,int(y2) / height ,int(x4) /width,int(y4) / height,int(x3) / width,int(y3) /height
        #print(filename, x1, y1, x2, y2, x4, y4,x3, y3)

        test(x,y,cx,cy,theta)

        file_object.write(str(class_name_to_id_mapping[name]) + ' '+ str(x1)+' '+str(y1)+' '+str(x2)+' '+str(y2)+' '+str(x4)+' '+str(y4)+' '+str(x3)+' '+str(y3))
        file_object.write('\n')
    file_object.close()

# Train the model

To train and validate the model, make sure your dataset is organized in the following structure under your working directory:
``` bash
datasets/
├── images/
│ ├── train/    # Training images
│ └── val/      # Validation images
├── labels/
│ ├── train/    # Corresponding YOLO-format label files for training images
│ └── val/      # Corresponding YOLO-format label files for validation images
```

In [ ]:
from ultralytics import YOLO


# Initialize model
model = YOLO('./runs/obb/BerlinHU-T22Without2/weights/best.pt')

# Train the model
model.train(data='./validBerlinHU.yaml', epochs=1300, name = 'BerlinHU-T22Without', batch=8)

# Validate the model

The validation result will save in colab file ./BerlinHUYoloStomata/ultralytics/runs/obb/val


In [ ]:
from ultralytics import YOLO


model = YOLO('./runs/obb/BerlinHU-T22Without2/weights/best.pt')

metric = model.val(data = './validBerlinHU.yaml')


print("mAP: ",metric.box.map)         # mAP@0.5:0.95
print("mAP50: ",metric.box.map50)      # mAP@0.5
print("f1 score: ",metric.box.f1)         # F1-scored
